# **Fairytale writing system**

## Abstract

This project develops a fairy-tale generation system by adapting a pretrained large language model to produce child bed time stories that follow a strict response format and satisfy user-specified constraints. The approach combines Domain-Adaptive Pretraining (DAPT) on a curated corpus of public-domain fairy-tales from Project Gutenberg to improve domain fluency and stylistic fidelity, and Supervised Fine-Tuning (SFT) using Low-Rank Adaptation (LoRA) to enforce consistent instruction following and output structure.

## 1. Introduction

Large language models can generate creative narratives; however, general-purpose instruction models frequently exhibit variability in adherence to structured output requirements and compliance with explicit constraints. In the context of children’s storytelling, such variability is problematic because users expect stable formatting, predictable tone, and safe, age-appropriate content. Furthermore, fairy-tale quality depends on domain-specific conventions that may not be reliably reproduced by a model trained broadly on heterogeneous text.

To address these limitations, the project adopts a two-stage adaptation strategy. First, DAPT aligns the model’s language distribution more closely with the fairy-tale domain. Second, SFT (implemented with LoRA for parameter efficiency) teaches the model to transform structured user inputs into consistently formatted outputs, improving controllability without requiring full-parameter fine-tuning.

This project is based on Qwen3-8B, a dense, decoder-only Transformer model from the Qwen3 family. It was chosen over other ~8B open weight models because it best matches the needs of a fairytale-writing system: it produces more natural storybook narration (strong creative/roleplay behavior), follows style and structure constraints more reliably (age level, moral, length, recurring phrases), supports longer contexts for better plot continuity, and is multilingual-friendly for writing/translation. On the practical side, its Apache-2.0 license is also easier for deployment and commercialization than more restrictive community/custom licenses, reducing legal and operational risk.

Qwen3 models follow the standard GPT-style causal language modeling setup: given a sequence of tokens, the model predicts the next token autoregressively. In Transformers, the Qwen3 architecture is implemented as a stack of decoder blocks with modern efficiency upgrades such as RMSNorm, RoPE positional embeddings, SwiGLU feed-forward layers, and grouped-query attention (GQA)

Key features:
- Superior human preference alignment, excelling in creative writing, role-playing, multi-turn dialogues, and instruction following, to deliver a more natural, engaging, and immersive conversational experience.
- Support of 100+ languages and dialects with strong capabilities for multilingual instruction following and translation.
- Smaller models in the ~2B range can be effective for short-form text or narrow tasks, but often struggle to maintain both creativity and coherence while also meeting multiple explicit constraints.
- Context length - 32k tokens natively

Architecture:
- **Token embedding layer** - It converts each token id into a dense vector. Embeddings let the model learn semantic relationships between tokens.
- **RoPE (Rotary Position Embeddings)** - it injects position information directly into attention by rotating query/key vectors based  on token position. Helps the model understand word order and supports long-context behavior better than older absolute position embeddings.
- **Transformer decoder blocks**
  - RMSNorm (normalization) -Normalizes activations to keep training stable.
  - Causal self-attention (with a causal mask) - Each token attends only to previous tokens (not future ones), enforcing autoregressive generation. It is the core mechanism that lets the model look back at context to decide the next token.
  - GQA (Grouped-Query Attention) - Uses more query heads than key/value heads (shared KV across groups of queries). It reduces KV-cache memory and speeds inference
  - Residual connection - Adds the block input back to the block output. This helps gradients flow during training and allows deep stacks to train reliably.
  - Feed-Forward Network (MLP) with SwiGLU
  - Final normalization + LM head - Final RMSNorm and a linear projection (LM head) convert hidden states into logits over the vocabulary. It produces the next-token probability distribution.



## 2. Objectives
2.1 Primary objective

Develop a fairy-tale generator that reliably produces high-quality, child-appropriate stories from structured user inputs while maintaining a stable and verifiable response format.

2.2 Evaluation objectives

Evaluate the final model’s narrative quality using an LLM-based judge with a structured rubric (craft/coherence and element integration), and compare the scores against stories sampled from the TinyStories dataset as a reference.

## 3. Methodology
###3.1 Data collection

A public-domain English corpus is constructed from Project Gutenberg, selected to emphasize fairy-tale and folklore texts. For the SFT stage, the TinyStories dataset is used to teach instruction-following and constraint adherence in short, child-friendly stories.

###3.2 Data cleaning

The Gutenberg dataset was cleaned from transcriber notes, duplicated sections, headers, footers.This step aims to maximize training signal quality by preserving story content while minimizing non-narrative noise.

The TinyStories dataset was also filtered to keep only examples with a complete prompt structure (summary, features, and required words), excluding entries with missing elements

###3.3 Token-safe chunking

Because books exceed the model’s training context window, texts are segmented into token-budgeted chunks. Chunking is performed at paragraph boundaries where possible to preserve semantic coherence and avoid splitting within words. A limited overlap between consecutive chunks is introduced to maintain local continuity.

###3.4 Domain-Adaptive Pretraining (DAPT)

DAPT is used to adapt the model’s distribution toward fairy-tale language and narrative patterns before instruction tuning. While the base model is generally fluent, domain adaptation improves stylistic consistency and reduces generic, modern, or off-tone phrasing.


###3.5 Supervised Fine-Tuning (SFT) with LoRA
SFT is applied to enforce controllability: structured prompting, consistent section formatting, and robust inclusion of user constraints. Because full-parameter fine-tuning of an 8B model is resource-intensive and increases overfitting risk, LoRA is used.

LoRA freezes the base model and trains only small low-rank adapter parameters inserted into selected layers - attention and MLP projection matrices.


## 4. Evaluation

Narrative quality is evaluated with an LLM judge (GPT-5 mini) that assigns rubric scores to each story. GPT-5 mini is used because it can follow a fixed rubric reliably and return consistent numeric ratings at low overhead, which makes it practical to score many samples.

The evaluation follows the multi-axis grading approach described in the writing benchmark: stories are scored on craft & coherence questions and element-integration questions that check whether required constraints are used naturally, not just mentioned. These rubric dimensions are then combined into a single score using a 60/40 weighted power mean (Hölder mean, p = 0.5), which behaves like a soft-minimum so weaknesses pull the score down more than strengths can compensate.

The judge also compares the final model’s generated stories against stories sampled from the TinyStories-style dataset, providing a reference point for whether the model’s outputs are comparable in clarity, coherence, and constraint use to the data distribution it learned from. To reduce bias, the same judge model and rubric are used for both sources, and stories are evaluated without revealing whether they are generated or dataset samples.

### Environment setup and imports

In [ ]:
!pip -q install -U pip packaging

!pip -q install openai pydantic

!pip -q install -U transformers trl datasets peft accelerate bitsandbytes sentencepiece evaluate rouge-score bert-score mauve-text


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

!pip -q uninstall -y flash-attn
!pip -q install -U ninja
!pip -q install -U flash-attn --no-build-isolation

  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os, re, json, math, random
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from typing import List, Dict, Optional, Tuple
from pathlib import Path

import torch
import pandas as pd
import numpy as np

from datasets import Dataset, load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig

from peft import PeftModel, LoraConfig, get_peft_model


from collections import Counter

from google.colab import drive
drive.mount("/content/drive")

from pydantic import BaseModel, Field
from openai import OpenAI
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
client = OpenAI()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
try:
    import flash_attn
    HAS_FA2 = True
except Exception:
    HAS_FA2 = False

if torch.cuda.is_available():
    GPU_MAJOR = torch.cuda.get_device_capability()[0]
else:
    GPU_MAJOR = 0

ATTN_IMPL = "flash_attention_2" if (HAS_FA2 and GPU_MAJOR >= 8) else "sdpa"
print("ATTN_IMPL =", ATTN_IMPL)


ATTN_IMPL = flash_attention_2


### Reproducibility
Training and evaluation include stochastic components like dataset shuffling, dropout, and probabilistic decoding during generation. Without controlling the random seed, experimental outcomes may vary across runs due solely to random variation rather than genuine differences between model variants.

In [ ]:
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

## Configurations

- One LoRA adapter for DAPT and SFT training is chosen. It keeps the parameter-efficient adaptation path consistent across stages and prevents additional confounding factors.

- The maximum sequence length is set to 2048 tokens as a practical trade-off between modeling capacity and computational feasibility. This context window is sufficient to accommodate the system prompt, user constraints, and structured output while still leaving enough room for generating approximately 1000 tokens of story text. Increasing the lenght will, the model will consume more VRAM and the training will be slower.


-  The rank r = 16 to provide sufficient adaptation capacity for DAPT, which targets a genuine distribution shift toward fairy-tale style and content. If r is too small, the adapter may not have enough degrees of freedom to represent the necessary domain-specific changes.

- The scaling factor α = 32 so that α/r equals 2, which constrains the update magnitude relative to the base weights and supports stable optimization.


- LoRA is applied to both attention and feed-forward (MLP) projections because these components contribute differently to the model’s behavior. Adapting attention projections (q, k, v, o) modifies how the model retrieves and integrates contextual information, which is relevant for maintaining narrative coherence, tracking entities, and sustaining repeated motives. Adapting MLP projections (gate, up, down) modifies the transformation of internal representations and is closely tied to stylistic realization, lexical choice, and the overall narrative of the generated text.

- A maximum generation budget of approximately 1000 new tokens is used to meet the requirement for long-form stories.

- Sampling with temperature 0.8 and top-p 0.9 provides a balance between diversity and coherence: it allows variation in narrative expression without excessively increasing the probability of incoherent or off-format outputs. Decreasing the temperature may cause repetition and reduce creativity. Reducing the top-p may reduce randomness and cause sudden shift in narratives.

- Top_k sampling to the 50 most likely next tokens at each step. It keeps the model from picking very low-probability tokens, which often causes random, unrealistic events, while still leaving enough options to avoid bland, repetitive phrasing. With children’s stories, reducing top_k can sound stiff; increasing top_k can drift.

- no_repeat_ngram blocks the model from repeating any exact 4-word sequence. Using 4 is a sweet spot: it stops obvious copy-paste phrasing without breaking natural repetition.

- repetition_penalty slightly reduces the probability of tokens that have already appeared, nudging the model away from repeating the same words while not hurting coherence. If the penalty is increasing, the model starts avoiding necessary words and becomes unnatural. Value of 1.06 is the lowest that removes the repetition without degrading style.


In [ ]:
MODEL_ID = "Qwen/Qwen3-8B"

MAX_SEQ_LEN = 2048

#Dataset
N_TOTAL = 50_000
EVAL_FRAC = 0.02

CHUNK_TARGET_TOKENS = 900

# Eval split sizes:
#DAPT_EVAL_FRACTION = 0.01
#SFT_EVAL_FRACTION  = 0.05


LORA_RANK   = 16
LORA_ALPHA  = 32


LORA_TARGET_MODULES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

GEN_STORY = dict(
    max_new_tokens=900,
    min_new_tokens=0,
    do_sample=True,
    temperature=0.8,
    top_p=0.9,   #0.85
    top_k = 50,
    no_repeat_ngram_size=4,
    repetition_penalty=1.06,
)


# Filepaths for saving into Google Drive
DRIVE_ROOT = "/content/drive/MyDrive/Colab Notebooks/fairytale_writer"
SAVE_DIR = os.path.join(DRIVE_ROOT, "data", "tinystories_sft_50k")

DAPT_RUN_DIR       = os.path.join(DRIVE_ROOT, "runs", "qwen3_dapt_lora")
DAPT_ADAPTER_DIR   = os.path.join(DRIVE_ROOT, "adapters", "qwen3_dapt_lora_adapter")


SFT_RUN_DIR        = os.path.join(DRIVE_ROOT, "runs", "qwen3_sft_lora")
FINAL_ADAPTER_DIR  = os.path.join(DRIVE_ROOT, "adapters", "qwen3_final_writer_lora_adapter")

# Create folders
for p in [DAPT_RUN_DIR, DAPT_ADAPTER_DIR, SFT_RUN_DIR, FINAL_ADAPTER_DIR]:
    os.makedirs(p, exist_ok=True)

print("DAPT_RUN_DIR:", DAPT_RUN_DIR)
print("DAPT_ADAPTER_DIR:", DAPT_ADAPTER_DIR)
print("SFT_RUN_DIR:", SFT_RUN_DIR)
print("FINAL_ADAPTER_DIR:", FINAL_ADAPTER_DIR)

DAPT_RUN_DIR: /content/drive/MyDrive/Colab Notebooks/fairytale_writer/runs/qwen3_dapt_lora
DAPT_ADAPTER_DIR: /content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter
SFT_RUN_DIR: /content/drive/MyDrive/Colab Notebooks/fairytale_writer/runs/qwen3_sft_lora
FINAL_ADAPTER_DIR: /content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter


## Main functions

All key length constraints in the pipeline—such as chunking, maximum sequence length, and the generation budget are specified in tokens rather than characters or words. Therefore, a tokenizer is required to measure and control these budgets consistently.

Many decoder-only models do not provide a dedicated padding token, but padding is required for batching, so the end-of-sequence token is reused as pad_token. During training, right padding is used because it aligns sequences in a way that is safer with packing and loss masking. During generation, left padding is used so the most recent tokens are aligned across the batch, which works better for batched decoding with variable-length prompts, especially when using the KV cache.


BF16 is preferred over FP16 because it retains a wider exponent range, which reduces the likelihood of numerical overflow and instability. This is particularly important for long sequence lengths, FP16 can more easily produce NaNs or divergent loss.

Gradient checkpointing is enabled to manage GPU memory usage. For an 8B-parameter model with long contexts, activation memory can exceed available VRAM. Checkpointing trades additional compute for substantially lower memory consumption, allowing the intended sequence length and effective batch configuration to remain feasible.

LoRA is used instead of full fine-tuning because it updates a small set of adapter parameters rather than the full model weights. This reduces computational and memory requirements, lowers the risk of catastrophic forgetting of general language ability.

Even though training happens on the GPU, initially has to read weights from disk and stage them in CPU memory before they’re placed on the GPU. For an 8B model, that CPU loading spike can be large enough to crash the runtime. Thats why *low_cpu_mem_usage* is enabled

Flash Attention 2 is used because it makes attention computation both faster and more memory-efficient. In normal (“eager”) attention, the model effectively forms a full
$𝑁×𝑁$ attention matrix for a sequence of length
𝑁, which is expensive to compute and costs a lot of GPU memory. Flash Attention 2 avoids materializing that full matrix by computing attention in fused, GPU-optimized tiles and streaming intermediate results. This reduces memory traffic and peak VRAM usage, which usually translates into higher training throughput (more tokens per second) and fewer memory bottlenecks compared to normal attention—especially at longer context lengths.

During training the key–value (KV) attention cache is disabled because it wastes GPU memory and can interfere with gradient checkpointing; training already processes full sequences with backprop, so caching past states provides no benefit. During inference the cache is enabled because generation happens token by token, and reusing the stored keys and values avoids recomputing attention over the whole prompt each step, making decoding much faster.

In [ ]:
def load_tokenizer(mode: str):
    tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True, use_fast=True)

    if tok.pad_token is None:
        tok.pad_token = tok.eos_token


    #right padding is safer for training
    tok.padding_side = "right" if mode == "train" else "left"
    return tok


def load_base_model(mode: str):
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map={"": 0},
        torch_dtype=torch.bfloat16,
        attn_implementation=ATTN_IMPL,
        low_cpu_mem_usage=True,
    )

    if mode == "train":
        model.train()
        model.config.use_cache = False
        model.gradient_checkpointing_enable()
    else:
        model.eval()
        model.config.use_cache = True

    return model


def add_lora(model):
    cfg = LoraConfig(
        r=LORA_RANK,
        lora_alpha=LORA_ALPHA,
        bias="none",
        target_modules=LORA_TARGET_MODULES,
        task_type="CAUSAL_LM",
    )
    return get_peft_model(model, cfg)


def load_for_dapt_training():
    tok = load_tokenizer("train")
    model = add_lora(load_base_model("train"))
    return model, tok


def load_for_sft_training_from_dapt(dapt_adapter_dir: str):
    tok = load_tokenizer("train")
    base = load_base_model("train")
    model = PeftModel.from_pretrained(base, dapt_adapter_dir, is_trainable=True)
    model.train()
    model.config.use_cache = False
    return model, tok


def load_eval_base():
    tok = load_tokenizer("eval")
    model = load_base_model("eval")
    return model, tok


def load_eval_with_adapter(adapter_dir: str):
    tok = load_tokenizer("eval")
    base = load_base_model("eval")
    model = PeftModel.from_pretrained(base, adapter_dir)
    model.eval()
    return model, tok







## Loading DAPT dataset

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/fairytale_writer/data/fairytales_clean.json"

with open(path, "r", encoding="utf-8") as f:
    records = json.load(f)

print("Loaded:", len(records))

Loaded: 500


In [ ]:
print("First record keys:", records[0].keys())
print("First title:", records[0].get("title"))
print("First text preview:\n", records[0].get("text", "")[:5000])

First record keys: dict_keys(['title', 'text'])
First title: Grimms' Fairy Tales
First text preview:
 Grimms’ Fairy Tales

By Jacob Grimm and Wilhelm Grimm

PREPARER’S NOTE

The text is based on translations from
the Grimms’ Kinder und Hausmärchen by
Edgar Taylor and Marian Edwardes.

CONTENTS:

THE GOLDEN BIRD
HANS IN LUCK
JORINDA AND JORINDEL
THE TRAVELLING MUSICIANS
OLD SULTAN
THE STRAW, THE COAL, AND THE BEAN
BRIAR ROSE
THE DOG AND THE SPARROW
THE TWELVE DANCING PRINCESSES
THE FISHERMAN AND HIS WIFE
THE WILLOW-WREN AND THE BEAR
THE FROG-PRINCE
CAT AND MOUSE IN PARTNERSHIP
THE GOOSE-GIRL
THE ADVENTURES OF CHANTICLEER AND PARTLET
1. HOW THEY WENT TO THE MOUNTAINS TO EAT NUTS
2. HOW CHANTICLEER AND PARTLET WENT TO VISIT MR KORBES
RAPUNZEL
FUNDEVOGEL
THE VALIANT LITTLE TAILOR
HANSEL AND GRETEL
THE MOUSE, THE BIRD, AND THE SAUSAGE
MOTHER HOLLE
LITTLE RED-CAP [LITTLE RED RIDING HOOD]
THE ROBBER BRIDEGROOM
TOM THUMB
RUMPELSTILTSKIN
CLEVER GRETEL
THE OLD MAN AND HIS GRANDSON
THE LITTLE PEA

## Chunking

Chunking is used because the raw texts (fairytales / books) are usually longer than the model’s max sequence length, and transformers can only train on fixed-length windows. Chunking turns long documents into many well-formed training examples the model can actually learn from.

Chunking into training examples by accumulating paragraphs until a token budget is reached, using the model tokenizer for token counting. This avoids splitting inside words and prefers paragraph/sentence boundaries for coherence. Setting the chunk token budget below the training max sequence length to leave room for the instruction prompt, and apply small overlap between consecutive chunks to preserve narrative continuity.

##### Architecture
- Spliting the daset into paragraphs using blank lines.

- Tokenizing each paragraph one at a time to get its token count.

- Keeping adding paragraphs into a current chunk until target_tokens is reached.

- When budget tokens is exceed finalize the current chunk and start new one with overlapping.

In [ ]:
def token_count(tok, text: str) -> int:
    return len(tok(text, add_special_tokens=False).input_ids)

def split_paragraphs(text: str) -> List[str]:
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    return [p.strip() for p in re.split(r"\n\s*\n+", text) if p.strip()]

def chunk_text_by_tokens(
    tok,
    text: str,
    target_tokens: int = 900,
    overlap_paragraphs: int = 1,
    min_tokens: int = 200,
    max_paragraph_tokens: int = 600,
) -> List[str]:
    paragraphs = split_paragraphs(text)
    chunks: List[str] = []
    current: List[str] = []
    current_tokens = 0


    def buffr():
      """
      buffers paragraphs until a chunk reaches a minimum token length
      """
        nonlocal current, current_tokens
        if not current:
            return
        chunk = "\n\n".join(current).strip()
        if token_count(tok, chunk) >= min_tokens:
            chunks.append(chunk)
        current, current_tokens = [], 0


    for p in paragraphs:
        p_tokens = token_count(tok, p)

        # Split very long paragraphs into sentences
        if p_tokens > max_paragraph_tokens:
            sentences = re.split(r"(?<=[.!?])\s+", p)
            buf = ""
            for s in sentences:
                candidate = (buf + " " + s).strip()
                if buf and token_count(tok, candidate) > max_paragraph_tokens:
                    idx = paragraphs.index(p)
                    paragraphs.insert(idx + 1, s)
                    p = buf
                    p_tokens = token_count(tok, p)
                    break
                buf = candidate

        if current and (current_tokens + p_tokens > target_tokens):
            buffer()

            if overlap_paragraphs > 0 and chunks:
                prev = split_paragraphs(chunks[-1])
                current = prev[-overlap_paragraphs:]
                current_tokens = token_count(tok, "\n\n".join(current))

        current.append(p)
        current_tokens += p_tokens

    buffer()
    return chunks


## Load DAPT dataset

In [ ]:
tok = load_tokenizer(mode="train")

chunk_rows = []
for r in records:
    title = r.get("title", "")
    text = r.get("text", "")
    if not text:
        continue

    for c in chunk_text_by_tokens(tok, text, target_tokens=CHUNK_TARGET_TOKENS):
        chunk_rows.append({"title": title, "text": c})

dapt_ds = Dataset.from_list(chunk_rows).shuffle(seed=42)
dapt_split = dapt_ds.train_test_split(test_size=DAPT_EVAL_FRACTION, seed=42)

dapt_train = dapt_split["train"]
dapt_eval  = dapt_split["test"]

print("Total chunks:", len(dapt_ds))
print("Train:", len(dapt_train), "Eval:", len(dapt_eval))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Total chunks: 51100
Train: 50589 Eval: 511


## DAPT training

- Packing reduces wasted padding tokens by concatenating examples into full-length sequences. Also increases tokens processed per step, improving throughput and often stabilizing optimizer statistics.

- A micro-batch size of 1 is used because an 8B model with ~2048-token sequences can exceed GPU memory when using larger per-device batches. Gradient accumulation over 8 steps increases the effective batch size without increasing peak VRAM usage. This makes the update closer to a true batch of 8 sequences, while staying within memory constraints.

- adamw_torch_fused is used because it keeps the AdamW optimization algorithm but uses PyTorch’s fused implementation, which is designed to be faster via kernel fusion.

- Peak learning rate is set to 1e-4 for domain-adaptive continued pretraining (DAPT) using with a decay schedule with warmup. A learning rate around 1e-4 is used in prior work for continued/domain-adaptive pretraining, including studies that explicitly report 1e-4 as the initial LR with cosine or linear scheduling.

- cosine decay provides a smoother LR reduction near the end of training than linear decay, often improving stability and convergence.

- Training for a single epoch is chosen because the dataset contains a large number of chunks, so one pass already yields a substantial number of updates.

In [ ]:
dapt_args = SFTConfig(
    output_dir=DAPT_RUN_DIR,
    max_length,
    packing=True,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,

    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.10,

    optim="adamw_torch_fused"

    num_train_epochs=1,

    logging_steps=30,
    logging_strategy="steps",
    save_steps=800,
    eval_strategy="no",
    save_total_limit=3,

    bf16=True,
    fp16=False,
    report_to="none",
    disable_tqdm=False, #progress bar active
)

model_dapt, tok_dapt = load_for_dapt_training()



m = model_dapt.get_base_model() if hasattr(model_dapt, "get_base_model") else model_dapt
val = getattr(m.config, "_attn_implementation", "MISSING_ATTR")
print("config attention:", hasattr(m.config, "_attn_implementation"))
print("Model config _attn_implementation =", repr(val))


trainer_dapt = SFTTrainer(
    model=model_dapt,
    args=dapt_args,
    train_dataset=dapt_train,
    processing_class=tok_dapt,
    formatting_func=lambda ex: ex["text"],
)

trainer_dapt.train()


trainer_dapt.model.save_pretrained(DAPT_ADAPTER_DIR)
tok_dapt.save_pretrained(DAPT_ADAPTER_DIR)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

config attention: True
Model config _attn_implementation = 'flash_attention_2'


Applying formatting function to train dataset:   0%|          | 0/50589 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/50589 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50589 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/50589 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
30,2.447900
60,2.324500
90,2.332600
120,2.284700
150,2.291000
180,2.264500
210,2.295200
240,2.275800
270,2.286400
300,2.260500


('/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/chat_template.jinja',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_dapt_lora_adapter/tokenizer.json')

After around 1000 steps the loss stays close to ~2.20 and only changes a little. Possible reasons:

- Diminishing returns: Early in DAPT the model learns the biggest differences in the new text (common words and style) very fast. Later improvements are smaller, so the loss stops dropping much.

- LoRA has limited capacity: Only the LoRA adapter is trained, not the full model. LoRA can adapt style and vocabulary, but it can reach a limit sooner than full fine-tuning. Increasing the LORA rank can give the adapter more ability to learn the new style.

- Packing adds variation: With packing, each batch is a different mix of text pieces. Some batches are easier and some are harder, so the loss can bounce around even if training is still slowly improving.


Data quality / mixed styles: The Gutenberg corpus can include mixed writing styles and some leftover noise, which can cause the loss cannot go much lower without more cleaning or more capacity.

## SFT with LoRA (instruction + format) on top of DAPT


## Build SFT dataset

In [ ]:


DATASET_ID = "roneneldan/TinyStoriesInstruct"
N_TOTAL = 150_000

train_stream_lines = load_dataset(DATASET_ID, split="train", streaming=True)

def iter_story_blocks(line_stream, end_token="<|endoftext|>"):
    buf = []
    for ex in line_stream:
        line = ex["text"]
        buf.append(line)
        if end_token in line:
            yield {"text": "\n".join(buf).strip()}
            buf = []

def take_n_blocks(block_iter, n):
    out = []
    for i, ex in enumerate(block_iter):
        if i >= n:
            break
        out.append(ex)
    return out

block_iter = iter_story_blocks(train_stream_lines)
raw_blocks = take_n_blocks(block_iter, N_TOTAL)




In [ ]:
print("Loaded story blocks:", len(raw_blocks))
print("Preview:\n", raw_blocks[49999]["text"][:3800])

Loaded story blocks: 50000
Preview:
 Summary: Benny the bunny becomes a hero when he stops a thief from stealing carrots in the park.
Words: spring, thief, adorable
Story: Once upon a time, there was a little bunny named Benny. Benny was very cute and everyone who saw him thought he was adorable. One day, Benny was playing in the park when he noticed a sneaky thief. The thief was trying to steal some carrots from the garden.
Benny didn't know what to do, but he knew he had to stop the thief. So, he ran as fast as he could and sprang into action. He chased the thief away and saved the carrots. All the animals in the park were very grateful to Benny for being so brave.
From that day on, Benny became known as the hero of the park. He was happy that he could help his friends and make them feel safe. Benny learned that even though he was small, he could do big things if he tried his best.
<|endoftext|>


The parser takes one raw text block and tries to split it into the parts it needs: Summary, Features, Words, and the actual Story. First it removes the <|endoftext|> token and cleans spaces. It looks for lines that start with Summary:, Features:, Words:, and Story:. When it finds Story:, it takes everything after that as the story text. If any of these parts are missing, or the story ends up empty, that example gets skipped. At the end it returns only examples that contains full templete. This istep is needed to filter the dataset and provide only full stories for better training.

In [ ]:


def parser(block: str):

    block = block.replace("<|endoftext|>", "").strip()
    if not block:
        return None

    words = features = summary = None
    story_lines = None

    lines = block.splitlines()
    for i, line in enumerate(lines):
        if line.startswith("Words:"):
            words = line[len("Words:"):].strip()
        elif line.startswith("Features:"):
            features = line[len("Features:"):].strip()
        elif line.startswith("Summary:"):
            summary = line[len("Summary:"):].strip()
        elif line.startswith("Story:"):
            # story is everything AFTER "Story:" line
            story_lines = lines[i+1:]
            break

    if not (words and features and summary and story_lines is not None):
        return None

    story = "\n".join(story_lines).strip()
    if not story:
        return None

    prompt = f"Summary: {summary}\nFeatures: {features}\nWords: {words}\nStory:"
    return prompt, story

rows = []
dropped = 0

for ex in raw_blocks:
    parsed = parser(ex["text"])
    if parsed is None:
        dropped += 1
        continue
    prompt, story = parsed
    rows.append({"text": f"{prompt}\n{story}"})

ds = Dataset.from_list(rows)



In [ ]:
print("Kept:", len(ds), "Dropped:", dropped)
print(ds[10000]["text"][:2400])

Kept: 46287 Dropped: 103713
Summary: Ben and Lily play a game of throwing and catching a ball, but when Lily gets hurt, Ben declines the prize of a hug and instead chooses to continue playing and make his friend happy. They switch to a different game and have fun together.
Features: Dialogue
Words: need, prize, weak
Story:
Ben and Lily are friends. They like to play games. Today they play a game with a ball. They throw the ball to each other and catch it. Whoever drops the ball is out.
Ben throws the ball to Lily. Lily catches it. Lily throws the ball to Ben. Ben catches it. They do this many times. They are good at the game.
But then Ben throws the ball too hard. Lily tries to catch it, but it is too fast. The ball hits her hand and falls to the ground. Lily says, "Ow! That hurt!"
Ben says, "I'm sorry, Lily. Are you okay?"
Lily says, "Yes, I'm okay. But you win the game. You get the prize."
Ben says, "What prize?"
Lily says, "The prize is a hug. You can hug me if you want."
Ben says, 

In [ ]:

os.makedirs(SAVE_DIR, exist_ok=True)

ds.save_to_disk(SAVE_DIR)
print("Saved dataset to:", SAVE_DIR)

In [ ]:

ds = load_from_disk(SAVE_DIR)
print(ds)
print(ds[0]["text"][:3400])

Dataset({
    features: ['text'],
    num_rows: 46287
})
Summary: Sara and Ben were playing in the park, but Sara wanted to go home because it was cold and dark. Ben convinced her to stay and play, but eventually agreed to go home and have hot cocoa.
Features: Dialogue
Words: quit, oak, gloomy
Story:
Sara and Ben were playing in the park. They liked to climb the big oak tree and pretend they were birds. They made nests with leaves and twigs and sang songs.
But today, the sky was gloomy and the wind was cold. Sara felt sad and cold. She wanted to go home and have some hot cocoa.
"Ben, I want to quit," she said. "It's too cold and dark. Let's go home."
Ben looked at Sara and frowned. He liked the oak tree and the park. He wanted to stay and play.
"No, Sara, don't quit," he said. "It's fun here. Look, there's a squirrel. Let's chase it."
Sara shook her head. She didn't want to chase the squirrel. She wanted to go home and have some hot cocoa.
"Please, Ben, let's go home," she said. "We ca

In [ ]:
# splitting

split = ds.train_test_split(test_size=EVAL_FRAC, seed=42)
sft_train = split["train"]
sft_eval  = split["test"]

print("Train:", len(sft_train))
print("Eval :", len(sft_eval))


Train: 45361
Eval : 926


### SFT training

- SFT updates are driven by supervised instruction targets. A lower learning rate than DAPT (5e-5) is used because SFT is intended to make a targeted behavioral adjustment—improving schema and constraint compliance—while preserving the broader domain adaptation learned during DAPT. Reducing the step size limits the magnitude of updates to the adapter and therefore lowers the risk of overwriting domain-specific stylistic knowledge, or memorizing scaffold phrasing instead of generalizing the structure. A high learning rate can make the model overfit those patterns quickly and drift in bad ways.


- For SFT max_length=1024 is used instead of 2048 mostly because of VRAM limits. Longer context means much more memory and slower training. Also, TinyStories examples are usually not that long, so 1024 should be enough to fit the prompt + story without wasting a lot of compute.

In [ ]:
sft_args = SFTConfig(
    output_dir=SFT_RUN_DIR,
    max_length=1024,
    packing=True,

    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,

    learning_rate=5e-5,
    num_train_epochs=1,
    lr_scheduler_type="cosine",
    warmup_ratio=0.10,

    optim="adamw_torch_fused

    logging_steps=100,
    logging_strategy="steps",


    eval_strategy="no",


    save_steps=100,
    save_total_limit=7,


    bf16=True,
    fp16=False,
    report_to="none",
    disable_tqdm=False,
)

model_sft, tok_sft = load_for_sft_training_from_dapt(DAPT_ADAPTER_DIR)

trainer_sft = SFTTrainer(
    model=model_sft,
    args=sft_args,
    train_dataset=sft_train,
    processing_class=tok_sft,
    formatting_func=lambda ex: ex["text"],
)


train_result = trainer_sft.train()
print(train_result)

trainer_sft.model.save_pretrained(FINAL_ADAPTER_DIR)
tok_sft.save_pretrained(FINAL_ADAPTER_DIR)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
100,1.171700
200,1.061100
300,1.028200
400,1.019200
500,1.006000
600,0.999800


TrainOutput(global_step=685, training_loss=1.041462083802606, metrics={'train_runtime': 16995.1474, 'train_samples_per_second': 0.644, 'train_steps_per_second': 0.04, 'total_flos': 4.994338424216064e+17, 'train_loss': 1.041462083802606, 'entropy': 0.9906364991087439, 'num_tokens': 10935155.0, 'mean_token_accuracy': 0.712163402368197, 'epoch': 1.0})


('/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/chat_template.jinja',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/fairytale_writer/adapters/qwen3_final_writer_lora_adapter/tokenizer.json')

### Creating prompts and generating functions



In [ ]:
SYSTEM_PROMPT = """You are a master fairy-tale writer with a warm, human voice.

Write ONLY the story text. No headings, no bullet points, no explanations, no repeating the fields.

Follow the Summary closely, include all Features, and use every required Word naturally.

Writing rules:
- Show emotions through actions, dialogue, and sensory details (avoid "he felt..." / "they knew..." / "she learned...").
- Avoid template phrases: "Suddenly", "Just then", "Out of nowhere", "Ever since then", "The moral of the story is".
- New characters/objects must have a believable reason to appear.
- Make at least TWO required words affect the plot.
- Use varied sentence lengths and natural, child-friendly language.

Dialogue requirement:
- Include dialogue throughout the story (at least 8 lines of spoken dialogue).
- Keep dialogue short and realistic for children; use quotation marks.

Format:
- Past tense, third-person.
- Separate scenes with blank lines.
- End with a warm final image (no moral statement)."""


In [ ]:
def make_user_prompt(c: dict) -> str:
    summary = str(c.get("summary", "")).strip()
    features = str(c.get("features", "")).strip()
    words_raw = str(c.get("words", "")).strip()
    words = [w.strip() for w in words_raw.replace("\n", ",").split(",") if w.strip()]
    words_line = ", ".join(words)

    return f"""SUMMARY:
{summary}

FEATURES (include all):
{features}

WORDS (use all naturally; at least TWO must matter to what happens):
{words_line}

WRITE:
- 600–900 words.
- few short scenes/paragraph groups separated by blank lines.
- At least 8 lines of dialogue. Spread them across the story (not all at the end).
- Do not summarize; write moment-by-moment scenes.

STORY: """


In [ ]:
eval_prompts = [
    #  Hansel and Gretel
    {
        "summary": "Hansel and Gretel, left near the edge of a deep forest, try to find their way home as night falls. Following a trail of crumbs and clever clues, they stumble upon a tempting gingerbread cottage owned by a sweet-voiced old woman who is not what she seems. Using courage and teamwork, they outsmart her and escape, returning home wiser and safe.",
        "features": "classic fairy-tale tone, sibling teamwork, a dark enchanted forest, a magical candy cottage, a clever escape, warm satisfying ending",
        "words": "crumb, lantern, gingerbread, oven, whisper",
    },

    # Little Red Riding Hood
    {
        "summary": "Little Red Riding Hood sets off through the forest to bring a basket of treats to her grandmother. She meets a clever wolf who tricks her into revealing where she is going and races ahead to the cottage. With danger close, Red must stay calm, notice the signs, and—through help arriving at the right moment—make it safely home, wiser about listening to good advice.",
        "features": "classic fairy-tale tone, dark forest journey, a clever trickster villain, building tension, rescue or narrow escape, warm satisfying ending",
        "words": "basket, hood, path, growl, cottage",
    },

    # Cinderella
    {
        "summary": "Cinderella is treated unfairly at home but keeps her kindness. When an invitation to a royal ball arrives, a small bit of magic gives her a chance to go—so long as she returns before midnight. At the ball she leaves behind a glass slipper, and later she must choose courage and honesty when the search begins, leading to a hopeful new life.",
        "features": "classic fairy-tale tone, hardship to hope, magical helper, royal ball, midnight deadline, satisfying ending",
        "words": "slipper, pumpkin, midnight, carriage, ash",
    },

    #  map + lost star
    {
        "summary": "A child finds a hand-drawn map tucked inside an old library book and follows it after closing time. The clues lead up to a quiet rooftop where a lost star is waiting, dim and lonely. With gentle magic and a little bravery, the child helps the star return to the night sky, then heads home feeling peaceful and proud.",
        "features": "modern child protagonist, gentle magic, library setting, wonder, cozy bedtime tone",
        "words": "map, bookmark, glitter, rooftop, star",
    },

    # shy dragon + music festival
    {
        "summary": "A nervous young dragon wants to join a small music festival but worries their voice and flame are too loud. With supportive friends, the dragon practices controlling tiny embers and learns to sing softly in a steady melody. When it’s time to step onto the stage, the dragon finds a calm way to perform and ends the night smiling among new friends.",
        "features": "dragon, music festival, self-control, supportive friends, gentle humor, warm satisfying ending",
        "words": "melody, ember, stage, practice, smile",
    },
]



In [ ]:
def _model_device(model):
    m = model.get_base_model() if hasattr(model, "get_base_model") else model
    return next(m.parameters()).device

@torch.inference_mode()
def generate_story(model, tok, constraints: Dict, gen_cfg: Dict) -> str:
    prompt_text = make_user_prompt(constraints)
    inputs = tok(prompt_text, return_tensors="pt").to(_model_device(model))
    prompt_len = inputs["input_ids"].shape[-1]

    out_ids = model.generate(
        **inputs,
        max_new_tokens=gen_cfg["max_new_tokens"],
        min_new_tokens=gen_cfg.get("min_new_tokens", 0),
        do_sample=gen_cfg["do_sample"],
        temperature=gen_cfg["temperature"],
        top_p=gen_cfg["top_p"],
        top_k=gen_cfg.get("top_k", 0),
        repetition_penalty=gen_cfg.get("repetition_penalty", 1.0),
        no_repeat_ngram_size=gen_cfg.get("no_repeat_ngram_size", 0),
        pad_token_id=tok.pad_token_id if tok.pad_token_id is not None else tok.eos_token_id,
        eos_token_id=tok.eos_token_id,
    )


    story_ids = out_ids[0, prompt_len:]
    story = tok.decode(story_ids, skip_special_tokens=True).strip()
    return story

## Write stories

In [ ]:
sft_model, tok = load_eval_with_adapter(FINAL_ADAPTER_DIR)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
N = 5
generated_stories = []

for i, c in enumerate(eval_prompts):
    if i >= N:
        break
    story = generate_story(sft_model, tok, c, gen_cfg=GEN_STORY)
    generated_stories.append(story)

for i, s in enumerate(generated_stories, start=1):
    print("-"*80)
    print(f"STORY {i}")
    print("-"*80)
    print(s)
    print()

--------------------------------------------------------------------------------
STORY 1
--------------------------------------------------------------------------------
Hansel and Gretl are walking in the dark forest. They are scared but they hold hands. "Look," Hansel says, "there's a small house with a big window." "Let's go see it," Gretl says. She picks up a crumb and drops it on the ground. Hansel follows the crumb with his eyes. He sees more crumbs. He walks behind the crumb. It leads him to the door of the house. He knocks on the door. The door opens. A woman with a warm voice says, "Welcome! You're very brave to come this far." She smiles. Hansel and Gertrude step inside. The room is full of things they like. There are toys, books, and food. The food looks good. It smells good too. Hansel goes to the kitchen. He sees a big oven. He wants to touch it. He reaches out. But then he hears a whisper. It says, "Don't touch the oven, it's hot!" Hansel jumps back. He tells Gretl about 

## Evaluation

### LLM-as-judge

The project evaluates each generated story using GPT-5 mini as a rubric-based judge. GPT-5 mini is chosen because it is strong at following structured instructions and returning consistent numeric scores, while remaining efficient enough to score many samples.

The judge produces two groups of metrics: Craft & Coherence (Q1–Q8), which rate basics like character clarity, plot and setting clarity, ending satisfaction, voice consistency, and readability; and Element Integration (Q9), which checks whether the required summary, features, and required words are included naturally rather than name-dropped. These rubric scores are then combined into a single overall score using a 60/40 weighted Hölder (power) mean with p = 0.5, so weak dimensions pull the final score down more than strong dimensions can compensate, rewarding well-rounded stories.

In [ ]:
JUDGE_SYSTEM_PROMPT = """You are a simple, consistent rater of child-friendly stories (TinyStories style).
Be fair for simple writing.

Give 0.0–10.0 scores (0.1 steps). Output JSON ONLY.

Craft & coherence (Q1–Q8):
Q1 Character clarity (who wants what)
Q2 Plot clarity (beginning→middle→end makes sense)
Q3 Setting clarity (can imagine where/when)
Q4 Ending satisfaction (feels finished)
Q5 Originality (not too cliché)
Q6 Theme clarity (teamwork/bravery/etc. comes through)
Q7 Voice consistency (tone/POV stays consistent)
Q8 Readability (clear sentences, not too repetitive)

Element integration (Q9A–Q9J):
9A Summary adherence
9B Feature coverage
9C–9G Word integration for the required words in order (1..5)
9H–9J must be null

Return exactly:
{
  "q1_to_q8": {"Q1":..., "Q2":..., "Q3":..., "Q4":..., "Q5":..., "Q6":..., "Q7":..., "Q8":...},
  "q9": {"9A":..., "9B":..., "9C":..., "9D":..., "9E":..., "9F":..., "9G":..., "9H":null, "9I":null, "9J":null}
}
No other keys, no notes, no text.
"""

In [ ]:
def make_judge_user_prompt(ep: dict, story: str) -> str:
    words = [w.strip() for w in str(ep.get("words","")).split(",") if w.strip()]
    return f"""SKELETON:
Summary: {ep.get("summary","")}
Features: {ep.get("features","")}
Required words (in order): {", ".join(words)}

STORY:
{story.strip()}
"""


class Q1toQ8(BaseModel):
    Q1: float; Q2: float; Q3: float; Q4: float; Q5: float; Q6: float; Q7: float; Q8: float

class Q9(BaseModel):
    A9: Optional[float] = Field(default=None, alias="9A")
    B9: Optional[float] = Field(default=None, alias="9B")
    C9: Optional[float] = Field(default=None, alias="9C")
    D9: Optional[float] = Field(default=None, alias="9D")
    E9: Optional[float] = Field(default=None, alias="9E")
    F9: Optional[float] = Field(default=None, alias="9F")
    G9: Optional[float] = Field(default=None, alias="9G")
    H9: Optional[float] = Field(default=None, alias="9H")
    I9: Optional[float] = Field(default=None, alias="9I")
    J9: Optional[float] = Field(default=None, alias="9J")

    model_config = {"populate_by_name": True}

class JudgeOut(BaseModel):
    q1_to_q8: Q1toQ8
    q9: Q9

In [ ]:
def holder_mean(scores, weights, p=0.5):
    num = sum(w * (s ** p) for s, w in zip(scores, weights))
    den = sum(weights)
    return (num / den) ** (1.0 / p)

def final_story_score(j: JudgeOut, p=0.5) -> float:
    craft = [j.q1_to_q8.Q1, j.q1_to_q8.Q2, j.q1_to_q8.Q3, j.q1_to_q8.Q4,
             j.q1_to_q8.Q5, j.q1_to_q8.Q6, j.q1_to_q8.Q7, j.q1_to_q8.Q8]
    craft_w = [0.60/8] * 8

    q9 = j.q9
    elems_raw = [q9.A9, q9.B9, q9.C9, q9.D9, q9.E9, q9.F9, q9.G9, q9.H9, q9.I9, q9.J9]
    elems = [float(x) for x in elems_raw if x is not None]

    if elems:
        elem_w = [0.40/len(elems)] * len(elems)
        scores = craft + elems
        weights = craft_w + elem_w
    else:
        scores, weights = craft, craft_w

    return float(holder_mean(scores, weights, p=p))


In [ ]:


def judge_story(ep: dict, story: str) -> JudgeOut:
    user_prompt = make_judge_user_prompt(ep, story)
    resp = client.responses.parse(
        model="gpt-5-mini",
        instructions=JUDGE_SYSTEM_PROMPT,
        input=user_prompt,
        text_format=JudgeOut,

    )
    return resp.output_parsed

In [ ]:
scores = {}
for i in range(5):
    j = judge_story(eval_prompts[i], generated_stories[i])
    scores[f"story{i+1}"] = round(final_story_score(j, p=0.5), 2)

scores

{'story1': 5.81, 'story2': 5.51, 'story3': 5.95, 'story4': 7.68, 'story5': 7.1}

In [ ]:


idxs = random.sample(range(len(ds)), 5)

dataset_scores = {}
for k, idx in enumerate(idxs, start=1):
    ep = {
        "summary": ds[idx].get("summary", ""),
        "features": ds[idx].get("features", ""),
        "words": ds[idx].get("words", ""),
    }
    story = ds[idx]["text"]

    judged = judge_story(ep, story)
    dataset_scores[f"dataset_story{k}"] = round(final_story_score(judged, p=0.5), 2)

print("Sampled dataset indices:", idxs)
dataset_scores


Sampled dataset indices: [41905, 7296, 1639, 18024, 16049]


{'dataset_story1': 4.99,
 'dataset_story2': 8.81,
 'dataset_story3': 5.54,
 'dataset_story4': 6.55,
 'dataset_story5': 5.78}

## 5. Conclusion



The project delivers what it set out to build: a small-model fairy-tale writer that can reliably produce readable, child-friendly stories from a structured prompt and then rank them with an external judge. The evaluation results show a clear pattern. The model’s original bedtime stories score better because they match the TinyStories-style training set: simple scenes, warm tone, straightforward conflict, and easy placement of required words. The classic fairy-tale retellings score lower because famous stories come with strict expectations—specific plot beats, escalating tension, and a satisfying payoff. When any of those beats are rushed, missing, or only name-dropped, the rubric and especially the soft-minimum power-mean score penalizes the output heavily.

Classic fairy tales were included on purpose as a stress test, even though the main goal is to generate original bedtime stories under specific constraints. Well-known stories provide a familiar ground truth structure: they have expected plot beats, recognizable tone, and clear endings. By asking the model to generate within those classic frameworks while still obeying required words and features, the project checks whether the system can handle tighter narrative structure and higher expectations—not just produce a cozy story.

The comparison with sampled dataset stories reinforces theory that the system can generate good bedtime stories. The dataset can produce high scoring examples, but the fine-tuned model tends to generate closer to the dataset’s average voice, but less often reaching the strongest, most complete narratives. The most practical conclusion is that the system is already useful as a **bedtime story writer** tool, while classic fairy-tale retellings remain the hardest case.



## 6. References

https://huggingface.co/datasets/manu/project_gutenberg

https://huggingface.co/datasets/roneneldan/TinyStories

https://drive.google.com/drive/folders/1OlwArVoMwBihl3zg9vQAhgMjas9Cbiyi


https://arxiv.org/html/2504.09687v1

https://huggingface.co/blog/samuellimabraz/peft-methods

https://huggingface.co/Qwen/Qwen3-8B

https://www.datacamp.com/tutorial/fine-tuning-qwen3

https://huggingface.co/docs/trl/sft_trainer

https://blog.ivan.digital/finetuning-qwen3-with-lora-done-right-94d6343e1814

https://github.com/ivan-digital/llm-alignment/blob/master/Qwen3_Adapters_SoftPrompt_LoRA_KL_SFT_Workshop.executed.ipynb

https://github.com/lechmazur/writing/



